# How to Build a Shotlist

This notebook provides an example of how to build a shotlist using tools in `disruption_py` that you can run on one of the PSFC mfews machines/virtual machines.

There are three SQL databases that contain useful data: the "runs", "shots", and "summary" databases. Let's get these three databases as pandas dataframes. There's also an interface in the data_handler to get the list of shots in the "disruptions" database.

In [2]:
from disruption_py.database import create_cmod_handler

data_handler = create_cmod_handler()
runs_table = data_handler.query(f"select * from runs", use_pandas=True)
shots_table = data_handler.query(f"select * from shots", use_pandas=True)
summary_table = data_handler.query(f"select * from summary", use_pandas=True)
disruptions_shot_list = data_handler.get_disruption_shotlist()

print(runs_table)

/home/awang/repos/disruption-warning-db-workflow/disruption_py/database.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, self.conn)
/home/awang/repos/disruption-warning-db-workflow/disruption_py/database.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, self.conn)
/home/awang/repos/disruption-warning-db-workflow/disruption_py/database.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, self.conn)
/home/awang/repos/disrupt

      USERNAME                     ENTERED      RUN      BRIEF  DBKEY
0      COCHRAN  2000-03-21 10:17:53.640000      321              1149
1      slwalsh  2022-05-04 16:40:55.147000   220504  Not C-Mod   2888
2        TERRY  2022-05-06 11:06:05.243000   220506  Not C-Mod   2903
3         YLIN  2022-05-17 16:12:49.470000   220517  Not C-Mod   2904
4         YLIN  2022-05-18 09:27:46.133000   220518  Not C-Mod   2905
...        ...                         ...      ...        ...    ...
2263   WUKITCH  2022-04-08 13:21:51.373000  1220408  Not C-Mod   2879
2264   WUKITCH  2022-04-11 16:20:08.437000  1220411  Not C-Mod   2880
2265   WUKITCH  2022-04-12 08:50:02.517000  1220412  Not C-Mod   2881
2266  gdecrist  2022-04-19 13:03:06.600000  1220419  Not C-Mod   2884
2267   WUKITCH  2022-05-04 08:59:32.480000  1220504  Not C-Mod   2886

[2268 rows x 5 columns]


## Example of Filtering Shots
So one day, I wanted to gather data for all shots that are "OK", were done from 2010 onward, and weren't disruptive (i.e. in the disruptions database). The little snippet of code below filters the shots database to get the shots that meet these criteria and saves the shot numbers to a pickle file.

In [9]:

def filter_shots_table(shots_table, disruptions_shot_list):

    # Only keep shots that are "ok".
    shots_table = shots_table[shots_table.SHOT_OK == 1]

    # Only keep shots that are 2010 or later.
    shots_table = shots_table[shots_table.SHOT > 1000000000]

    # Remove shots that are in the disruption shot list.
    also_in_disruptions_shot_list = shots_table.SHOT.isin(disruptions_shot_list.shot.to_numpy())
    shots_table = shots_table[~also_in_disruptions_shot_list]
    return shots_table


filtered_shots_table = filter_shots_table(shots_table, disruptions_shot_list)
print(filtered_shots_table)
filtered_shots_table.SHOT.to_pickle("/tmp/shots_2010_and_later.pkl")

        USERNAME                     ENTERED      RUN        SHOT  \
14434  MDSSERVER  2000-03-14 10:03:06.857000  1000314  1000314001   
14435  MDSSERVER  2000-03-14 10:28:54.420000  1000314  1000314002   
14436  MDSSERVER  2000-03-20 08:35:43.890000  1000320  1000320001   
14437  MDSSERVER  2000-03-20 08:49:45.810000  1000320  1000320002   
14438  MDSSERVER  2000-03-20 09:01:19.470000  1000320  1000320003   
...          ...                         ...      ...         ...   
51814    mdsplus  2017-01-11 12:21:56.287000  1170111  1170111001   
51815    mdsplus  2017-01-11 13:08:46.900000  1170111  1170111002   
51816    mdsplus  2017-01-11 13:31:59.830000  1170111  1170111003   
51818    mdsplus  2017-01-12 08:22:18.400000  1170112  1170112001   
51819    mdsplus  2017-01-12 11:06:33.400000  1170112  1170112002   

              BRIEF PLASMA_SHOT    QUALITY_COMMENT  SHOT_OK  \
14434   no comment            T                ‰0      1.0   
14435   no comment            T              